In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne 

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "your key"
import openai
openai.api_key = "your key"

In [13]:
#!pip install llama-index-readers-youtube-transcript

In [6]:
#!pip install youtube_transcript_api

In [14]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
youtube_documents = loader.load_data(
    ytlinks=['https://www.youtube.com/watch?v=jITPOcBQQW8', 'https://www.youtube.com/watch?v=xFfnJhZeL_Y', 'https://www.youtube.com/watch?v=g_LNu6Aaxvk']
)

print(youtube_documents)

[Document(id_='jITPOcBQQW8', embedding=None, metadata={'video_id': 'jITPOcBQQW8'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[Music]\nwhat is a savannah cat a savannah cat is\na domestic cat breed that has a percent\nof african cerval wild blood\nthey are known to love water swimming\nrunning in the shower with you\nbeing extremely affectionate and bonded\nto their owners while still maintaining\nthe independence of a cat\nthey still use the litter box and lower\ngeneration savannahs can eat high\nquality regular cat food\nthey're known for extreme intelligence\nconstant purring\ntheir high energy playing fetch and\nbeing easily trainable for commands\nthese savannah cat qualities are more\nprevalent in the highest generations as\nthey have the most african circle of\nblood\nthe attraction to the savannah cat breed\ncomes from their dog-like personality\nand beautiful aesthetic\nthey are highly intelligent and can be\ntaught tricks just as 

In [15]:
from llama_index.core import VectorStoreIndex

vector_indices = {}
vector_query_engines = {}

breeds = ["Savanah", "Ragdol", "Maine Coon"]

for breed, youtube in zip(breeds, youtube_documents):
    vector_index = VectorStoreIndex.from_documents([youtube])
    query_engine = vector_index.as_query_engine(similarity_top_k=3)
    vector_indices[breed] = vector_index
    vector_query_engines[breed] = query_engine

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [16]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata


query_engine_tools = []
for breed in breeds:
    query_engine = vector_query_engines[breed]

    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=breed,
            description=f"Provides information about the cat breed {breed}"
        ),
    )
    query_engine_tools.append(query_engine_tool)

subquestion_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)    

In [17]:
response = subquestion_engine.query("Tell me the difference between Ragdoll and Maine Coon cat.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generated 2 sub questions.
[Ragdol] Q: What are the characteristics of Ragdoll cats?
[Maine Coon] Q: What are the unique features of Maine Coon cats?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[Ragdol] A: Ragdoll cats are known for their friendly and gentle personality. They are docile companions, not prone to aggression, and are very tolerant with people and other animals. Ragdol

In [18]:
print(response)

Ragdoll cats are known for their friendly and gentle personality, being docile and tolerant with people and other animals. They are affectionate and form strong bonds quickly, often described as "lap cats." On the other hand, Maine Coon cats have unique physical features such as tufts of fur between the paw pads and out of the ears, large ears with lynx tips, big muscles and solid bones, a long, double-layered, and thick coat, a big, bushy tail, and big, oval eyes in various colors.
